<a href="https://colab.research.google.com/github/arunsechergy/TSAI-DeepNLP-END/blob/main/assignments/assignment9/Assignment9_Model1_CoQA_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model v1


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd 
from torchtext.data import Example,Field, BucketIterator,Dataset

import spacy
import numpy as np

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
train=pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
test=pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json')


In [3]:
train['data'][0]['story']

'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of t

In [4]:
import time
def data_prep(dt):
  st=time.time()
  story=[]
  question=[]
  answer=[]
  for i in range(0,dt.data.shape[0]):
    sty=dt.data[i]['story']
    for j in range(0,len(dt.data[i]['questions'])):
      story.append(sty)
      question.append(dt.data[i]['questions'][j]['input_text'])
      answer.append(dt.data[i]['answers'][j]['input_text'])
  print(time.time()-st)
  return(story,question,answer)


In [ ]:
# import time
# def data_prep(dt):
#   st=time.time()
#   story=[]
#   question=[]
#   answer=[]
  
#   for i in range(0,dt.data.shape[0]):
#     sty=dt.data[i]['story']
#     story.append(sty)
#     q=[]
#     a=[]
#     for j in range(0,len(dt.data[i]['questions'])):
#       q.append(dt.data[i]['questions'][j]['input_text'])
#       q1=" ".join(q)
#       a.append(dt.data[i]['answers'][j]['input_text'])
#       a1=" ".join(a)
#     question.append(q1)
#     answer.append(a1)
#   print(time.time()-st)
#   return(story,question,answer)


In [5]:
st_train,question_train,answer_train=data_prep(train)

1.717503547668457


In [ ]:
st_train[0]

'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to be made available online. \n\nThe Vatican Secret Archives were separated from the library at the beginning of t

In [ ]:
question_train[0]

'When was the Vat formally opened? what is the library for? for what subjects? and? what was started in 2014? how do scholars divide the library? how many? what is the official name of the Vat? where is it? how many printed books does it contain? when were the Secret Archives moved from the rest of the library? how many items are in this secret collection? Can anyone use this library? what must be requested to view? what must be requested in person or by mail? of what books? What is the Vat the library of? How many books survived the Pre Lateran period? what is the point of the project started in 2014? what will this allow?'

In [6]:
st_train,question_train,answer_train=data_prep(train)
train_data=pd.DataFrame(
    {'story': st_train,
     'question': question_train,
     'answer': answer_train
    })

1.7009906768798828


In [7]:
train_data.shape

(108647, 3)

In [8]:
MAX_LEN = 100
train_data['story'] = train_data['story'].apply(lambda x: ' '.join(x.split(' ')[:MAX_LEN]))

In [10]:
train_data['story'][0]

'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their'

In [11]:
len(train_data['story'][0])

607

In [12]:
train_data['sty_ques']=train_data['story']+" "+train_data['question']

In [13]:
train_data.head()

,story,question,answer,sty_ques
0,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their",When was the Vat formally opened?,It was formally established in 1475,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their When was the Vat formally opened?"
1,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their",what is the library for?,research,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their what is the library for?"
2,"The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their",for what subjects?,"history, and law","The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their for what subjects?"
3,"The Vatican Apostolic Library (), more commonly called the Vatican 

In [14]:
st_test,question_test,answer_test=data_prep(test)
test_data=pd.DataFrame(
    {'story': st_test,
     'question': question_test,
     'answer': answer_test
    })

0.12848472595214844


In [15]:
test_data.shape

(7983, 3)

In [16]:
test_data['story'] = test_data['story'].apply(lambda x: " ".join(x.split(' ')[:MAX_LEN]))

In [17]:
test_data['sty_ques']=test_data['story']+" "+test_data['question']

In [18]:
test_data.head()

,story,question,answer,sty_ques
0,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being",What color was Cotton?,white,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being What color was Cotton?"
1,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being",Where did she live?,in a barn,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being Where did she live?"
2,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being",Did she live alone?,no,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being Did she live alone?"
3,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being",Who did she live with?,with her mommy and 5 sisters,"Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a n

## Obtaining only revelent fields from the data set

In [19]:
train_cols=train_data[['sty_ques','answer']]
test_cols=test_data[['sty_ques','answer']]

In [20]:
train_cols.shape

(108647, 2)

In [21]:
test_cols.shape

(7983, 2)

### Create our fields to process our data

In [22]:
question = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)
answer = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [23]:
fields = [('questions', question),('answers',answer)]

In [24]:
fields

[('questions', <torchtext.data.field.Field at 0x7fd2857af860>),
 ('answers', <torchtext.data.field.Field at 0x7fd2857bcf98>)]

In [25]:
example = [Example.fromlist([train_cols.sty_ques[i],train_cols.answer[i]], fields) for i in range(train_cols.shape[0])]

In [26]:
questionDataset = Dataset(example, fields)

In [27]:
#(train, valid) = questionDataset.split(split_ratio=[0.80, 0.20], random_state=random.seed(SEED))

In [28]:
train=questionDataset
print(len(train))

108647


In [29]:
print(vars(train.examples[0]))

{'questions': ['the', 'vatican', 'apostolic', 'library', '(', ')', ',', 'more', 'commonly', 'called', 'the', 'vatican', 'library', 'or', 'simply', 'the', 'vat', ',', 'is', 'the', 'library', 'of', 'the', 'holy', 'see', ',', 'located', 'in', 'vatican', 'city', '.', 'formally', 'established', 'in', '1475', ',', 'although', 'it', 'is', 'much', 'older', ',', 'it', 'is', 'one', 'of', 'the', 'oldest', 'libraries', 'in', 'the', 'world', 'and', 'contains', 'one', 'of', 'the', 'most', 'significant', 'collections', 'of', 'historical', 'texts', '.', 'it', 'has', '75,000', 'codices', 'from', 'throughout', 'history', ',', 'as', 'well', 'as', '1.1', 'million', 'printed', 'books', ',', 'which', 'include', 'some', '8,500', 'incunabula', '.', '\n\n', 'the', 'vatican', 'library', 'is', 'a', 'research', 'library', 'for', 'history', ',', 'law', ',', 'philosophy', ',', 'science', 'and', 'theology', '.', 'the', 'vatican', 'library', 'is', 'open', 'to', 'anyone', 'who', 'can', 'document', 'their', 'when', 'wa

In [30]:
print(vars(train.examples[0]))

{'questions': ['the', 'vatican', 'apostolic', 'library', '(', ')', ',', 'more', 'commonly', 'called', 'the', 'vatican', 'library', 'or', 'simply', 'the', 'vat', ',', 'is', 'the', 'library', 'of', 'the', 'holy', 'see', ',', 'located', 'in', 'vatican', 'city', '.', 'formally', 'established', 'in', '1475', ',', 'although', 'it', 'is', 'much', 'older', ',', 'it', 'is', 'one', 'of', 'the', 'oldest', 'libraries', 'in', 'the', 'world', 'and', 'contains', 'one', 'of', 'the', 'most', 'significant', 'collections', 'of', 'historical', 'texts', '.', 'it', 'has', '75,000', 'codices', 'from', 'throughout', 'history', ',', 'as', 'well', 'as', '1.1', 'million', 'printed', 'books', ',', 'which', 'include', 'some', '8,500', 'incunabula', '.', '\n\n', 'the', 'vatican', 'library', 'is', 'a', 'research', 'library', 'for', 'history', ',', 'law', ',', 'philosophy', ',', 'science', 'and', 'theology', '.', 'the', 'vatican', 'library', 'is', 'open', 'to', 'anyone', 'who', 'can', 'document', 'their', 'when', 'wa

In [31]:
example1 = [Example.fromlist([test_cols.sty_ques[i],test_cols.answer[i]], fields) for i in range(test_cols.shape[0])]
valid = Dataset(example1, fields)

In [32]:
len(valid)

7983

In [33]:
MAX_VOCAB_SIZE = 25_000

question.build_vocab(train,max_size = MAX_VOCAB_SIZE, 
                     vectors = "glove.6B.100d",unk_init = torch.Tensor.normal_,min_freq = 2)
answer.build_vocab(train,max_size = MAX_VOCAB_SIZE,
                     vectors = "glove.6B.100d",unk_init = torch.Tensor.normal_,min_freq = 2)


.vector_cache/glove.6B.zip: 862MB [06:31, 2.20MB/s]                           
100%|█████████▉| 399960/400000 [00:15<00:00, 26128.40it/s]

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
device

device(type='cuda')

In [36]:
BATCH_SIZE = 64

train_iterator, valid_iterator = BucketIterator.splits(
    (train, valid), 
    batch_size = BATCH_SIZE, 
    device = device,sort=False)

In [37]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [38]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]

        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [39]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [40]:
INPUT_DIM = len(question.vocab)
OUTPUT_DIM = len(answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [41]:
print(INPUT_DIM)
print(OUTPUT_DIM)

25004
15647


In [42]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(25004, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(15647, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=15647, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [43]:
optimizer = optim.Adam(model.parameters())

In [44]:
TRG_PAD_IDX = answer.vocab.stoi[answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [45]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.questions
        trg = batch.answers
              
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [46]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.questions
            trg = batch.answers

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [47]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [48]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

100%|█████████▉| 399960/400000 [00:30<00:00, 26128.40it/s]

Epoch: 01 | Time: 3m 48s
	Train Loss: 5.487 | Train PPL: 241.450
	 Val. Loss: 5.167 |  Val. PPL: 175.474
Epoch: 02 | Time: 3m 48s
	Train Loss: 4.937 | Train PPL: 139.283
	 Val. Loss: 4.657 |  Val. PPL: 105.289
Epoch: 03 | Time: 3m 47s
	Train Loss: 4.496 | Train PPL:  89.694
	 Val. Loss: 4.495 |  Val. PPL:  89.573
Epoch: 04 | Time: 3m 46s
	Train Loss: 4.156 | Train PPL:  63.796
	 Val. Loss: 4.474 |  Val. PPL:  87.730
Epoch: 05 | Time: 3m 48s
	Train Loss: 3.767 | Train PPL:  43.246
	 Val. Loss: 4.539 |  Val. PPL:  93.582
Epoch: 06 | Time: 3m 48s
	Train Loss: 3.347 | Train PPL:  28.413
	 Val. Loss: 4.697 |  Val. PPL: 109.575
Epoch: 07 | Time: 3m 48s
	Train Loss: 2.922 | Train PPL:  18.585
	 Val. Loss: 4.874 |  Val. PPL: 130.870
Epoch: 08 | Time: 3m 48s
	Train Loss: 2.592 | Train PPL:  13.352
	 Val. Loss: 5.031 |  Val. PPL: 153.038
Epoch: 09 | Time: 3m 49s
	Train Loss: 2.333 | Train PPL:  10.304
	 Val. Loss: 5.198 |  Val. PPL: 180.824
Epoch: 10 | Time: 3m 48s
	Train Loss: 2.152 | Train PPL

In [50]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

	 Test  Loss: 5.235 |  Test PPL : 210.7 

